In [1]:
import pandas as pd

**Import Mortality Table**

In [2]:
Commutation = pd.read_html("https://www.ssa.gov/oact/STATS/table4c6.html")

In [3]:
Commutation = Commutation[0].loc[:119]

In [4]:
Commutation

Exact age                 Male                                     \
    Exact age Death  probability a Number of  lives b Life expectancy   
0           0             0.006081             100000           76.22   
1           1             0.000425              99392           75.69   
2           2             0.000260              99350           74.72   
3           3             0.000194              99324           73.74   
4           4             0.000154              99305           72.76   
..        ...                  ...                ...             ...   
115       115             0.723732                  0            0.86   
116       116             0.759919                  0            0.80   
117       117             0.797915                  0            0.74   
118       118             0.837811                  0            0.68   
119       119             0.879701                  0            0.63   

                  Female                                      
    Death  probability a Number of  lives b Life  expectancy  
0               0.005046             100000            81.28  
1               0.000349              99495            80.69  
2               0.000212              99461            79.72  
3               0.000166              99440            78.73  
4               0.000137              99423            77.75  
..                   ...                ...              ...  
115             0.718435                  0             0.87  
116             0.759919                  0             0.80  
117             0.797915                  0             0.74  
118             0.837811                  0             0.68  
119             0.879701                  0             0.63  

[120 rows x 7 columns]

**Cleaning data**

In [5]:
Commutation.dtypes

Exact age  Exact age               object
Male       Death  probability a    object
           Number of  lives b      object
           Life expectancy         object
Female     Death  probability a    object
           Number of  lives b      object
           Life  expectancy        object
dtype: object

In [6]:
Commutation['Male'] = Commutation['Male'].astype({'Death  probability a': float,
                                                  'Number of  lives b': float,
                                                  'Life expectancy': float})

In [7]:
Commutation['Female'] = Commutation['Female'].astype({'Death  probability a': float,
                                                      'Number of  lives b': float,
                                                      'Life  expectancy': float})

In [8]:
Commutation['Exact age'] = Commutation['Exact age'].astype({'Exact age': int})

In [9]:
Commutation.dtypes

Exact age  Exact age                 int32
Male       Death  probability a    float64
           Number of  lives b      float64
           Life expectancy         float64
Female     Death  probability a    float64
           Number of  lives b      float64
           Life  expectancy        float64
dtype: object

In [10]:
Commutation.rename(columns = {'Death  probability a' : '$q_x$',
                              'Number of  lives b' : '$l_x$',
                              'Life expectancy' : '$e_x$',
                              'Life  expectancy' : '$e_x$',
                              'Exact age' : '$x$'
                             },level =1, inplace = True)

In [11]:
Commutation.set_index(Commutation['Exact age']['$x$'], inplace = True)

In [12]:
Commutation.drop(columns = 'Exact age', inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [13]:
Commutation

Male                     Female                 
        $q_x$     $l_x$  $e_x$     $q_x$     $l_x$  $e_x$
$x$                                                      
0    0.006081  100000.0  76.22  0.005046  100000.0  81.28
1    0.000425   99392.0  75.69  0.000349   99495.0  80.69
2    0.000260   99350.0  74.72  0.000212   99461.0  79.72
3    0.000194   99324.0  73.74  0.000166   99440.0  78.73
4    0.000154   99305.0  72.76  0.000137   99423.0  77.75
..        ...       ...    ...       ...       ...    ...
115  0.723732       0.0   0.86  0.718435       0.0   0.87
116  0.759919       0.0   0.80  0.759919       0.0   0.80
117  0.797915       0.0   0.74  0.797915       0.0   0.74
118  0.837811       0.0   0.68  0.837811       0.0   0.68
119  0.879701       0.0   0.63  0.879701       0.0   0.63

[120 rows x 6 columns]

**Define interest rate and discount factor**

In [14]:
i = 0.025
v = 1/(1+i)
d = i * v

**Create Commutation Table**

In [15]:
Commutation['Male', '$d_x$'] = Commutation['Male']['$q_x$'] * Commutation['Male']['$l_x$']
Commutation['Female', '$d_x$'] = Commutation['Female']['$q_x$'] * Commutation['Female']['$l_x$']

In [16]:
Commutation['$v^x$'] = v ** Commutation.index

In [17]:
Commutation['Male', '$D_x$'] = Commutation['$v^x$'] * Commutation['Male']['$l_x$']
Commutation['Female', '$D_x$'] = Commutation['$v^x$'] * Commutation['Female']['$l_x$']

In [18]:
Cx = []
for x in range(Commutation.shape[0]):
    if x < Commutation.shape[0] - 1:
        Cx.append(Commutation['Male', '$d_x$'].iloc[x] * Commutation['$v^x$'].iloc[x+1])
    else:
        Cx.append(Commutation['Male', '$d_x$'].iloc[x] * 0)
Commutation['Male', '$C_x$'] = Cx

Cx = []
for x in range(Commutation.shape[0]):
    if x < Commutation.shape[0] - 1:
        Cx.append(Commutation['Female', '$d_x$'].iloc[x] * Commutation['$v^x$'].iloc[x+1])
    else:
        Cx.append(Commutation['Female', '$d_x$'].iloc[x] * 0)
Commutation['Female', '$C_x$'] = Cx

In [19]:
Mx = []
for x in range(Commutation.shape[0]):
    Mx.append(Commutation['Male', '$C_x$'][x:].sum())
Commutation['Male', '$M_x$'] = Mx

Mx = []
for x in range(Commutation.shape[0]):
    Mx.append(Commutation['Female', '$C_x$'][x:].sum())
Commutation['Female', '$M_x$'] = Mx

In [20]:
Commutation['Male', '$A_x$'] = Commutation['Male']['$M_x$'] / Commutation['Male']['$D_x$']
Commutation['Female', '$A_x$'] = Commutation['Female']['$M_x$'] / Commutation['Female']['$D_x$']

In [21]:
Commutation['Male', '$a_x$'] = (1 - Commutation['$v^x$']) / d
Commutation['Female', '$a_x$'] = (1 - Commutation['$v^x$']) / d

In [22]:
Commutation['Male', '$A_x$'].fillna(1, inplace = True)
Commutation['Female', '$A_x$'].fillna(1, inplace = True)

In [23]:
Commutation

Male                     Female                         Male  \
        $q_x$     $l_x$  $e_x$     $q_x$     $l_x$  $e_x$       $d_x$   
$x$                                                                     
0    0.006081  100000.0  76.22  0.005046  100000.0  81.28  608.100000   
1    0.000425   99392.0  75.69  0.000349   99495.0  80.69   42.241600   
2    0.000260   99350.0  74.72  0.000212   99461.0  79.72   25.831000   
3    0.000194   99324.0  73.74  0.000166   99440.0  78.73   19.268856   
4    0.000154   99305.0  72.76  0.000137   99423.0  77.75   15.292970   
..        ...       ...    ...       ...       ...    ...         ...   
115  0.723732       0.0   0.86  0.718435       0.0   0.87    0.000000   
116  0.759919       0.0   0.80  0.759919       0.0   0.80    0.000000   
117  0.797915       0.0   0.74  0.797915       0.0   0.74    0.000000   
118  0.837811       0.0   0.68  0.837811       0.0   0.68    0.000000   
119  0.879701       0.0   0.63  0.879701       0.0   0.63    0.000000   

         Female     $v^x$           Male         Female        Male  \
          $d_x$                    $D_x$          $D_x$       $C_x$   
$x$                                                                   
0    504.600000  1.000000  100000.000000  100000.000000  593.268293   
1     34.723755  0.975610   96967.804878   97068.292683   40.206163   
2     21.085732  0.951814   94562.760262   94668.411660   23.986651   
3     16.507040  0.928599   92232.207890   92339.925422   17.456633   
4     13.620951  0.905951   89965.428782   90072.330958   13.516757   
..          ...       ...            ...            ...         ...   
115    0.000000  0.058446       0.000000       0.000000    0.000000   
116    0.000000  0.057021       0.000000       0.000000    0.000000   
117    0.000000  0.055630       0.000000       0.000000    0.000000   
118    0.000000  0.054273       0.000000       0.000000    0.000000   
119    0.000000  0.052949       0.000000       0.000000    0.000000   

         Female          Male        Female      Male    Female       Male  \
          $C_x$         $M_x$         $M_x$     $A_x$     $A_x$      $a_x$   
$x$                                                                          
0    492.292683  16860.502989  14616.195373  0.168605  0.146162   0.000000   
1     33.050570  16267.234697  14123.902690  0.167759  0.145505   1.000000   
2     19.580198  16227.028534  14090.852120  0.171601  0.148844   1.975610   
3     14.954564  16203.041882  14071.271922  0.175677  0.152386   2.927424   
4     12.038936  16185.585250  14056.317358  0.179909  0.156056   3.856024   
..          ...           ...           ...       ...       ...        ...   
115    0.000000      0.000000      0.000000  1.000000  1.000000  38.603710   
116    0.000000      0.000000      0.000000  1.000000  1.000000  38.662156   
117    0.000000      0.000000      0.000000  1.000000  1.000000  38.719177   
118    0.000000      0.000000      0.000000  1.000000  1.000000  38.774807   
119    0.000000      0.000000      0.000000  1.000000  1.000000  38.829080   

        Female  
         $a_x$  
$x$             
0     0.000000  
1     1.000000  
2     1.975610  
3     2.927424  
4     3.856024  
..         ...  
115  38.603710  
116  38.662156  
117  38.719177  
118  38.774807  
119  38.829080  

[120 rows x 19 columns]

In [24]:
Commutation = Commutation.reindex(['$v^x$','Male','Female'], level=0, axis =1)

In [25]:
Commutation

$v^x$      Male                                              \
                  $q_x$     $l_x$  $e_x$       $d_x$          $D_x$   
$x$                                                                   
0    1.000000  0.006081  100000.0  76.22  608.100000  100000.000000   
1    0.975610  0.000425   99392.0  75.69   42.241600   96967.804878   
2    0.951814  0.000260   99350.0  74.72   25.831000   94562.760262   
3    0.928599  0.000194   99324.0  73.74   19.268856   92232.207890   
4    0.905951  0.000154   99305.0  72.76   15.292970   89965.428782   
..        ...       ...       ...    ...         ...            ...   
115  0.058446  0.723732       0.0   0.86    0.000000       0.000000   
116  0.057021  0.759919       0.0   0.80    0.000000       0.000000   
117  0.055630  0.797915       0.0   0.74    0.000000       0.000000   
118  0.054273  0.837811       0.0   0.68    0.000000       0.000000   
119  0.052949  0.879701       0.0   0.63    0.000000       0.000000   

                                                      Female                   \
          $C_x$         $M_x$     $A_x$      $a_x$     $q_x$     $l_x$  $e_x$   
$x$                                                                             
0    593.268293  16860.502989  0.168605   0.000000  0.005046  100000.0  81.28   
1     40.206163  16267.234697  0.167759   1.000000  0.000349   99495.0  80.69   
2     23.986651  16227.028534  0.171601   1.975610  0.000212   99461.0  79.72   
3     17.456633  16203.041882  0.175677   2.927424  0.000166   99440.0  78.73   
4     13.516757  16185.585250  0.179909   3.856024  0.000137   99423.0  77.75   
..          ...           ...       ...        ...       ...       ...    ...   
115    0.000000      0.000000  1.000000  38.603710  0.718435       0.0   0.87   
116    0.000000      0.000000  1.000000  38.662156  0.759919       0.0   0.80   
117    0.000000      0.000000  1.000000  38.719177  0.797915       0.0   0.74   
118    0.000000      0.000000  1.000000  38.774807  0.837811       0.0   0.68   
119    0.000000      0.000000  1.000000  38.829080  0.879701       0.0   0.63   

                                                                               
          $d_x$          $D_x$       $C_x$         $M_x$     $A_x$      $a_x$  
$x$                                                                            
0    504.600000  100000.000000  492.292683  14616.195373  0.146162   0.000000  
1     34.723755   97068.292683   33.050570  14123.902690  0.145505   1.000000  
2     21.085732   94668.411660   19.580198  14090.852120  0.148844   1.975610  
3     16.507040   92339.925422   14.954564  14071.271922  0.152386   2.927424  
4     13.620951   90072.330958   12.038936  14056.317358  0.156056   3.856024  
..          ...            ...         ...           ...       ...        ...  
115    0.000000       0.000000    0.000000      0.000000  1.000000  38.603710  
116    0.000000       0.000000    0.000000      0.000000  1.000000  38.662156  
117    0.000000       0.000000    0.000000      0.000000  1.000000  38.719177  
118    0.000000       0.000000    0.000000      0.000000  1.000000  38.774807  
119    0.000000       0.000000    0.000000      0.000000  1.000000  38.829080  

[120 rows x 19 columns]

**Notes** \
$x$ : Age \
$i$ : Interest rate \
$v$ : Discount factor \
$q_x$ : Probability of dying between ages $x$ and $x + 1$ \
$l_x$ : Number surviving to age $x$ \
$d_x$ : Number dying between ages $x$ and $x + 1$ \
$e_x$ : Expectation of life at age $x$ \
$D_x$ : $v^x * l_x$ \
$C_x$ : $v^{x+1} * d_x$ \
$M_x$ : $\sum{C_{x+i}}$ where $i$ start from 0 \
$A_x$ : Present Value of 1 Compentation \
$a_x$ : Present Value of 1 Premium per year

**Gross Premium Valuation Reserve**

Masa pembayaran premi sama dengan masa pertanggungan

In [26]:
def GPV_Reserve(Comp, Premium, Persentase, n, t, x):
    def PV_Outcome(Comp, Premium, Persentase, n, t, x):
        def PV_Compentation(Comp, n, t, x): #n: usia, t: masa pertanggungan, x: cadangan tahun ke-
            for i in range(n,n+t+1):
                Mn = Commutation['Male', '$M_x$'].iloc[i]
                Mnt = Commutation['Male', '$M_x$'].iloc[n+t]
                Dn = Commutation['Male', '$D_x$'].iloc[i]
                Dnt = Commutation['Male', '$D_x$'].iloc[n+t]
                An = (Mn - Mnt + Dnt)/Dn
                if i == n + x:
                    return Comp * An

        def PV_Operational(Premium, Persentase, n, t, x): #n: usia, t: masa pertanggungan, x: cadangan tahun ke-
            for i in range(t,-1,-1):
                an = Commutation['Male', '$a_x$'].iloc[i]
                if i == t - x:
                    return Premium * Persentase * an
    
        return PV_Compentation(Comp, n, t, x) + PV_Operational(Premium, Persentase, n, t, x)

    def PV_Income(Premium, n, t, x): #n: usia, t: masa pertanggungan, x: cadangan tahun ke-
        for i in range(t,-1,-1):
            an = Commutation['Male', '$a_x$'].iloc[i]
            if i == t - x:
                return Premium * an

    return PV_Outcome(Comp, Premium, Persentase, n, t, x) - PV_Income(Premium, n, t, x)

In [27]:
print(GPV_Reserve(100000000,1000000,0.3,50,20,5))

62270219.87347412


In [28]:
print("GROSS PREMIUM VALUATION RESERVE CALCULATOR\n")

Comp = float(input("Amount of compentation: "))
Premium = float(input("Premium per year: "))
Operasional = float(input("Operational fee (in percentage) :"))
n = int(input("Policyholder age at the beginning of policy period: "))
t = int(input("Policy period (in year): "))
x = int(input("x-th year Gross Premium Valuation Reserve: "))

print(x, "-th year Gross Premium Valuation Reserve: ", GPV_Reserve(Comp, Premium, Operasional, n, t, x), sep = "")

GROSS PREMIUM VALUATION RESERVE CALCULATOR

Amount of compentation: 100000000
Premium per year: 1000000
Operational fee (in percentage) :0.3
Policyholder age at the beginning of policy period: 50
Policy period (in year): 20
x-th year Gross Premium Valuation Reserve: 5
5-th year Gross Premium Valuation Reserve: 62270219.87347412
